# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features
    features_sframe = data_sframe[features]
    features_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return (features_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [23]:
def predict_output(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [7]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [8]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [9]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [10]:
import math

In [11]:
def normalize_features(feature_matrix):
    norms = np.sqrt((feature_matrix**2).sum(axis=0))
    normalized_features = feature_matrix / norms
    return normalized_features, norms

To test the function, run the following:

In [13]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [132]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [133]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [134]:
simple_feature_matrix

array([[0.00680209, 0.00353021, 0.00583571],
       [0.00680209, 0.00768869, 0.00583571],
       [0.00680209, 0.00230361, 0.00389048],
       ...,
       [0.00680209, 0.00305154, 0.00389048],
       [0.00680209, 0.00478673, 0.00583571],
       [0.00680209, 0.00305154, 0.00389048]])

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [135]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [136]:
prediction = predict_output(simple_feature_matrix, weights)
print(prediction.reshape(-1,1))

[[0.02675867]
 [0.04339256]
 [0.01990703]
 ...
 [0.02289873]
 [0.03178473]
 [0.02289873]]


In [137]:
simple_feature_matrix*weights

array([[0.00680209, 0.01412086, 0.00583571],
       [0.00680209, 0.03075475, 0.00583571],
       [0.00680209, 0.00921446, 0.00389048],
       ...,
       [0.00680209, 0.01220617, 0.00389048],
       [0.00680209, 0.01914693, 0.00583571],
       [0.00680209, 0.01220617, 0.00389048]])

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [138]:
#ro = ((simple_feature_matrix*weights + (output-prediction).reshape(-1,1)) * simple_feature_matrix).sum(axis=0)

In [140]:
ro = []
for i in range(len(weights)):
    ro.append(np.sum(simple_feature_matrix[:,i] * ((output-prediction) + simple_feature_matrix[:,i] * weights[i])))

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [142]:
for r in ro:
    print(f"{r*2:e}")

1.588006e+08
1.758789e+08
1.619334e+08


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [64]:
ro

array([79400300.03492916, 87939470.77299108, 80966698.67596565])

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [86]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:,i] * ((output-prediction) + feature_matrix[:,i] * weights[i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [87]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [143]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    maxchange = np.inf
    n = len(initial_weights)
    weights = initial_weights.copy()
    
    while maxchange >= tolerance:
        maxchange = 0
        for i in range(n):
            old_weight = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            maxchange = max(maxchange, abs(old_weight-weights[i]))
        print(maxchange)
        
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [144]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [145]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [146]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

79400304.65805088
9138168.3664971
8194809.409369811
6598905.038273297
5522173.194295086
4621129.8103132695
3867108.1062483788
3236118.8106070757
2708086.9395514876
2266213.1093986705
1896439.062647283
1587000.403191924
1328052.3109535538
1111356.3280020133
930018.2512413897
778268.7926896214
651280.0290381983
545011.8008177392
456083.1743439846
381664.8769945614
319389.2836340554
267275.0903974995
223664.2793214321
187169.36834619567
156629.26844373718
131072.34346086532
109685.49742341787
91788.30581157655
76811.36778943986
64278.19066403061
53790.030225310475
45013.20466164127
37668.4784415178
31522.178408872336
26378.759449586272
22074.583205301315
18472.711903650314
15458.551670111716
12936.206713106483
10825.428390417248
9059.062091000378
7580.910704761744
6343.946706216782
5308.815969310701
4442.5857121907175
3717.696737717837
3111.0866348855197
2603.45604621619
2178.6546567231417
1823.16737023741
1525.6843252405524
1276.7410706579685
1068.4174534380436
894.0856380611658
748.1992

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [147]:
def compute_rss_weights(data, output, weights):
    rss = np.sum((output-predict_output(data,weights))**2)
    return rss

In [148]:
compute_rss_weights(normalized_simple_feature_matrix, output, weights)

1630492481484487.8

In [150]:
print(weights)
print(simple_features)

[21624998.36636292 63157246.78545421        0.        ]
['sqft_living', 'bedrooms']


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [159]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [160]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [161]:
train_data_array,train_output = get_numpy_data(train_data,all_features,'price')
test_data_array,test_output = get_numpy_data(test_data,all_features,'price')

train_data_normalized,train_data_norms = normalize_features(train_data_array)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [162]:
initial_weights = np.zeros(len(all_features)+1)
weights1e7 = lasso_cyclical_coordinate_descent(train_data_normalized, train_output, initial_weights, 1e7, 1)

71114625.75280938
5024356.368689939
5012324.913952567
5000676.4208620265
4991243.173139848
4984361.343354017
4979504.929205112
4973806.565254949
2879222.9993843436
2706150.766220279
2532295.6486418396
2368316.5778220594
2213124.3506660387
1552443.6001314223
507838.0331589952
438814.37059121206
371479.6226081252
313415.1055806726
264244.8321387619
222753.86079965532
187770.6673064679
158280.0933571905
133420.90160612762
112465.98829612508
94802.21345191076
79912.6874505356
67361.6926195696
56781.942492034286
47863.83575203642
40346.39661949128
34009.6378518343
28668.123146481812
24165.540612939745
20370.128526780754
17170.819508824497
14473.990294985473
12200.721983704716
10284.490585509688
8669.22029242292
7307.6424985565245
6159.91255100444
5192.443752352148
4376.924493331462
3689.4897535815835
3110.022725433111
2621.5661239661276
2209.8259559720755
1862.753226429224
1570.191341623664
1323.5788908191025
1115.6991086155176
940.468686543405
792.7597535327077
668.2498160935938
563.295266

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [163]:
print(weights1e7)
print(np.array(all_features)[abs(weights1e7[1:]) > 0])

[24429600.60933314        0.                0.         48389174.35227978
        0.                0.          3317511.16271982  7329961.9848964
        0.                0.                0.                0.
        0.                0.        ]
['sqft_living' 'waterfront' 'view']


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [164]:
weights1e8 = lasso_cyclical_coordinate_descent(train_data_normalized, train_output, initial_weights, 1e8, 1)

71114625.75280938
0


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [165]:
print(weights1e8)
print(np.array(all_features)[abs(weights1e8[1:]) > 0])

[71114625.75280938        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]
[]


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [166]:
weights1e4 = lasso_cyclical_coordinate_descent(train_data_normalized, train_output, initial_weights, 1e4, 5e5)

71114625.75280938
8464015.947381664
6818451.358692966
5570165.219010256
4559838.467593711
3732056.6971487477
3053211.613525532
2497517.435575299
2043868.9732462838
1674717.7090572938
1375420.756921731
1296621.814295642
1257340.9360978305
1228271.2481895015
1206854.3647149876
1191059.4059796147
1179030.770174943
1169777.4334484115
1162340.675885938
1155980.529007785
1150139.679658614
1144406.327518247
1138483.9180533811
1132166.5716905221
1125319.167779088
1117861.2488759682
1109754.0628223121
1100990.1790671349
1091585.2124428377
1081571.2676139101
1070991.7840532064
1059897.5170069262
1048343.4363107383
1036386.3636026233
1024083.2006756812
1011489.6284699365
998659.1783918813
985642.5960174277
972487.4324027151
959237.8107269779
945934.3262659311
932614.046117574
919310.5819878355
906054.2149560824
892872.055697456
879109.3022427484
866038.2667416856
853191.449429512
840521.7810123041
828028.4039622545
815717.9208187759
803598.2063784003
791676.7696929872
779960.2228539735
768454.139

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [167]:
print(weights1e4)
print(np.array(all_features)[abs(weights1e4[1:]) > 0])

[ 77779073.91265225 -22884012.25023361  15348487.08089996
  92166869.69883074  -2139328.0824278   -8818455.54409492
   6494209.73310655   7065162.05053198   4119079.21006765
  18436483.52618776 -14566678.54514342  -5528348.75179426
 -83591746.20730537   2784276.46012858]
['bedrooms' 'bathrooms' 'sqft_living' 'sqft_lot' 'floors' 'waterfront'
 'view' 'condition' 'grade' 'sqft_above' 'sqft_basement' 'yr_built'
 'yr_renovated']


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [168]:
weights1e4_normalized = weights1e4/train_data_norms
weights1e7_normalized = weights1e7/train_data_norms
weights1e8_normalized = weights1e8/train_data_norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [169]:
print(weights1e7_normalized[3])

161.3174562483786


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [170]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [171]:
print(compute_rss_weights(test_feature_matrix,test_output,weights1e4_normalized))
print(compute_rss_weights(test_feature_matrix,test_output,weights1e7_normalized))
print(compute_rss_weights(test_feature_matrix,test_output,weights1e8_normalized))

227781004760225.34
275962079909185.28
537166150034084.9


***QUIZ QUESTION***

Which model performed best on the test data?